In [7]:
from openai import OpenAI
import os

client = OpenAI(base_url=os.getenv("URL"), api_key=os.getenv("KEY"))

resp = client.chat.completions.create(
    messages = [{ 
        "role": "user", 
        "content": "Should I trade JK Dobbins and CD Lamb for Lamar Jackson and Tyrone Tracy Jr in Fantasy Football? You have access to 2 tools. One is called get_stats and can give you stats for a player, you must specify the name of the player, the last N games you want, and which stats you want. Another tool is called get_rank, and takes in a name and league type and gives you a value amount for that player from 0-100. If you need access to any of those tools, please return the requests in a list with the given parameters and I will respond with the results. I want the most up to date info, which includes games up to November 23 2024." 
    }],
    model = "meta-llama/Meta-Llama-3.1-8B-Instruct",
    temperature=0)
print(resp.choices[0].message.content)

To help you make an informed decision, I'll use the tools to gather information about the players involved in the trade. Here are the requests I'd like to make:

1. **get_stats** for JK Dobbins, last 8 games, and the following stats: rushing yards, rushing touchdowns, receptions, receiving yards, and receiving touchdowns.
2. **get_stats** for CD Lamb, last 8 games, and the following stats: receptions, receiving yards, and receiving touchdowns.
3. **get_stats** for Lamar Jackson, last 8 games, and the following stats: passing yards, passing touchdowns, rushing yards, and rushing touchdowns.
4. **get_stats** for Tyrone Tracy Jr, last 8 games, and the following stats: receptions, receiving yards, and receiving touchdowns.
5. **get_rank** for JK Dobbins, and the league type is PPR (Points Per Reception).
6. **get_rank** for CD Lamb, and the league type is PPR (Points Per Reception).
7. **get_rank** for Lamar Jackson, and the league type is PPR (Points Per Reception).
8. **get_rank** for Ty

In [27]:
from typing import List
from datasets import load_dataset
from datetime import date, time, datetime
import dataclasses


@dataclasses.dataclass
class Flight:
    id: int
    date: date
    airline: str
    flight_number: str
    origin: str
    destination: str
    departure_time: time
    arrival_time: time
    available_seats: int


def parse_flight(flight):
    return Flight(
        id=flight["id"],
        date=datetime.strptime(flight["date"], "%Y-%m-%d").date(),
        airline=flight["airline"],
        flight_number=flight["flight_number"],
        origin=flight["origin"],
        destination=flight["destination"],
        departure_time=datetime.strptime(flight["departure_time"], "%H:%M").time(),
        arrival_time=datetime.strptime(flight["arrival_time"], "%H:%M").time(),
        available_seats=flight["available_seats"],
    )


def load_flights_dataset() -> List[Flight]:
    return [
        parse_flight(flight)
        for flight in load_dataset("nuprl/llm-systems-flights", split="train")
    ]

In [28]:
# print out dataset to file
flights = load_flights_dataset()
with open("flights.txt", "w") as f:
    for flight in flights:
        f.write(f"{flight.id}, {flight.date}, {flight.airline}, {flight.flight_number}, {flight.origin}, {flight.destination}, {flight.departure_time}, {flight.arrival_time}, {flight.available_seats}\n")

# - prompt: I need to go from ATL to SEA on Jan 9
#   expected_type: find-flights
#   expected_result: [663, 664, 665]
# - prompt: Book the earliest flight for the journey
#   expected_type: book-flight
#   expected_result: 663



In [31]:
from typing import List, Optional

flights = load_flights_dataset()

def find_flights(origin: str, destination: str, date: date) -> List[Flight]:
        flights_two = []

        for flight in flights:
            if flight.origin == origin and flight.destination == destination and flight.date == date:
                flights_two.append(flight)

        return flights_two

def book_flight(flight_id: int) -> Optional[int]:
        for flight in flights:
            if flight.id == flight_id:
                return flight.id

        return None

['find-flights', [Flight(id=662, date=datetime.date(2023, 1, 9), airline='United', flight_number='UA6982', origin='ATL', destination='SEA', departure_time=datetime.time(1, 48), arrival_time=datetime.time(3, 48), available_seats=102), Flight(id=663, date=datetime.date(2023, 1, 9), airline='JetBlue', flight_number='B68202', origin='ATL', destination='SEA', departure_time=datetime.time(9, 55), arrival_time=datetime.time(11, 55), available_seats=103), Flight(id=664, date=datetime.date(2023, 1, 9), airline='Southwest', flight_number='WN4071', origin='ATL', destination='SEA', departure_time=datetime.time(20, 23), arrival_time=datetime.time(22, 23), available_seats=107)]]


In [ ]:
from typing import List, Optional

class Agent:

    # The complete conversation with the LLM, including the system prompt.
    conversation: List[dict]
    # The formatted response from the last tool call.
    text_prefix: Optional[str]
    # The current database of flights. The tools update this database.
    flights: List[Flight]
    client: OpenAI
    # Global variables used in tool calls.
    program_state: dict

    def find_flights(self, origin: str, destination: str, date: date) -> List[Flight]:
        pass
        
    def book_flight(self, flight_id: int) -> Optional[int]:
        pass